### 0: Imports

In [1]:
import datetime
import pandas as pd
import praw
from pmaw import PushshiftAPI
import mysql.connector
from mysql.connector import Error

### 1: API Infos

In [2]:
info_file = open("api info.txt", "r")
id = info_file.readline().replace("\n", "")
secret = info_file.readline().replace("\n", "")
ua = info_file.readline().replace("\n", "")

### 2: Initialisation

In [69]:
reddit = praw.Reddit(
 client_id=id,
 client_secret=secret,
 user_agent=ua
)
api_praw = PushshiftAPI(praw=reddit)

before = int(datetime.datetime(2023,3,28,0,0).timestamp())
after = int(datetime.datetime(2015,3,28,0,0).timestamp())

### 3: Fetching Data

In [71]:
posts = api_praw.search_submissions(subreddit="france", limit=None, before=before, after=after)
post_list = [post for post in posts]

KeyboardInterrupt: 

### 4: DB functions

In [6]:
def create_db_connection(host, user, password, db):
    connection = None
    try:
        connection = mysql.connector.connect(
            host = host,
            user = user,
            passwd = password,
            database = db
        )
        print("MySQL Database connection established")
    except Error as err:
        print(f"Error: '{err}'")

    return connection

def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
    except Error as err:
        print(f"Error: '{err}'")

### 5: DB connection

In [37]:
connection = create_db_connection("localhost", "root", "######", "######") # Connect to the Database

MySQL Database connection successful


### 6: Scraping posts

In [38]:
for x in post_list:
    ins = f"INSERT INTO post VALUES (\"{x['id']}\", {x['over_18']}, {str(x['num_comments'] > 0)}, {str(x['selftext'] == '[deleted]')},\"{x['title']}\", \"{x['selftext']}\");"
    execute_query(connection, ins)

Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful


### 7: Scraping comments

In [ ]:
def explore_and_insert_comments(commentForest, postID):
    for comment in commentForest:
        replies = comment.replies
        parent = comment.parent_id
        if parent.startswith('t3_'):
            ins = f"INSERT INTO comments VALUES (\"{comment.id}\", {str(len(replies) > 0)}, \"{comment.body}\", {postID}, NULL);"
        else:
            ins = f"INSERT INTO comments VALUES (\"{comment.id}\", {str(len(replies) > 0)}, \"{comment.body}\", {postID}, {parent.replace('t1_', '')} );"
        execute_query(connection, ins)
        if (len(replies) > 0):
            explore_and_insert_comments(replies, postID)


In [63]:
id = [x['id'] for x in post_list]
id_formated = [i if i.startswith('t3_') else f't3_{i}' for i in id]

n = 0
for sub in reddit.info(fullnames=id_formated):
    explore_and_insert_comments(sub.comments, id[n])
    n += 1

101


### 8: Closing DB connection

In [39]:
connection.close()